In [233]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm as tqdm

%matplotlib inline

import torch
import torchvision

import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

from torch.utils.data import Dataset, DataLoader

In [234]:
m = 100
size = 500
desired_num = size + 1000  

In [235]:
np.random.seed(1)

y = np.concatenate((np.zeros(500),np.ones(500),np.ones(500)*2,np.ones(500)*3,np.ones(500)*4,
                    np.ones(500)*5,np.ones(500)*6,np.ones(500)*7,np.ones(500)*8,np.ones(500)*9))
#y = np.random.randint(0,3,6000)
idx= []
for i in range(10):
    print(i,sum(y==i))
    idx.append(y==i)

In [236]:
x = np.zeros((5000,2))

np.random.seed(12)
x[idx[0],:] = np.random.multivariate_normal(mean = [5,5],cov=[[0.1,0],[0,0.1]],size=sum(idx[0]))
x[idx[1],:] = np.random.multivariate_normal(mean = [-6,7],cov=[[0.1,0],[0,0.1]],size=sum(idx[1]))
x[idx[2],:] = np.random.multivariate_normal(mean = [-5,-4],cov=[[0.1,0],[0,0.1]],size=sum(idx[2]))
x[idx[3],:] = np.random.multivariate_normal(mean = [-1,0],cov=[[0.1,0],[0,0.1]],size=sum(idx[3]))
x[idx[4],:] = np.random.multivariate_normal(mean = [0,2],cov=[[0.1,0],[0,0.1]],size=sum(idx[4]))
x[idx[5],:] = np.random.multivariate_normal(mean = [1,0],cov=[[0.1,0],[0,0.1]],size=sum(idx[5]))
x[idx[6],:] = np.random.multivariate_normal(mean = [0,-1],cov=[[0.1,0],[0,0.1]],size=sum(idx[6]))
x[idx[7],:] = np.random.multivariate_normal(mean = [0,0],cov=[[0.1,0],[0,0.1]],size=sum(idx[7]))
x[idx[8],:] = np.random.multivariate_normal(mean = [-0.5,-0.5],cov=[[0.1,0],[0,0.1]],size=sum(idx[8]))
x[idx[9],:] = np.random.multivariate_normal(mean = [0.4,0.2],cov=[[0.1,0],[0,0.1]],size=sum(idx[9]))

In [237]:
for i in range(10):
    plt.scatter(x[idx[i],0],x[idx[i],1],label="class_"+str(i))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [238]:
mean_x = np.mean(x,axis=0,keepdims=True)
std_x = np.std(x,axis=0,keepdims=True) 
x = ( x - mean_x  )  / std_x

In [239]:
for i in range(10):
    plt.scatter(x[idx[i],0],x[idx[i],1],label="class_"+str(i))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [240]:
foreground_classes = {'class_0','class_1' }

background_classes = {'bg_classes',}

In [241]:
mosaic_list_of_images =[]
mosaic_label = []
fore_idx=[]
for j in tqdm(range(desired_num)):
    np.random.seed(j)
    fg_class  = np.random.randint(0,3)
    fg_idx = np.random.randint(0,m)
    a = []
    for i in range(m):
        if i == fg_idx:
            b = np.random.choice(np.where(idx[fg_class]==True)[0],size=1)
            a.append(x[b])
#             print("foreground "+str(fg_class)+" present at " + str(fg_idx))
        else:
            bg_class = np.random.randint(3,10)
            b = np.random.choice(np.where(idx[bg_class]==True)[0],size=1)
            a.append(x[b])
#             print("background "+str(bg_class)+" present at " + str(i))
    a = np.concatenate(a,axis=0)
    mosaic_list_of_images.append(np.reshape(a,(m,2)))
    mosaic_label.append(fg_class)
    fore_idx.append(fg_idx)

In [242]:
len(mosaic_list_of_images), mosaic_list_of_images[0],mosaic_list_of_images[0].shape

In [243]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list_of_images, mosaic_label, fore_idx):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list_of_images
    self.label = mosaic_label
    self.fore_idx = fore_idx

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx], self.fore_idx[idx]


In [244]:
batch = 250
msd1 = MosaicDataset(mosaic_list_of_images[0:size], mosaic_label[0:size] , fore_idx[0:size])
train_loader = DataLoader( msd1 ,batch_size= batch ,shuffle=True) 


batch = 250
msd2 = MosaicDataset(mosaic_list_of_images[size:], mosaic_label[size:] , fore_idx[size:])
test_loader = DataLoader( msd2 ,batch_size= batch ,shuffle=False)

**My cross Entropy loss**

In [245]:
def my_cross_entropy(output,target,loss_criterion):    
    batch = output.size(0)
    #print(batch)
    patches = output.size(1)
    classes = output.size(2)    
    output = torch.reshape(output,(batch*patches,classes))
    target = target.repeat_interleave(patches)    
    loss = loss_criterion(output,target)
    #print(final_loss)
    return loss

In [246]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(2, 3)
    #self.fc2 = nn.Linear(50,3)
    torch.nn.init.xavier_normal_(self.fc1.weight)
    torch.nn.init.zeros_(self.fc1.bias)
    #torch.nn.init.xavier_normal_(self.fc2.weight)
    #torch.nn.init.zeros_(self.fc2.bias)
  def forward(self, x):
    #x = F.relu(self.fc1(x))
    x = self.fc1(x)
    # print(x.shape)
    return x

In [257]:
torch.manual_seed(13)
net = Net().double()
net = net.to("cuda")

In [258]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)#, momentum=0.9)

In [259]:
acti = []
loss_curi = []

epochs  = 300
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data1 in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels,_ = data1
        #print(labels.shape)
        inputs = inputs.double()
        labels = labels.long()
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs.shape,inputs.shape)
        loss = my_cross_entropy(outputs, labels,criterion)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        mini = 2
        if i % mini== (mini-1):    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / mini))
            ep_lossi.append(running_loss/mini) # loss per minibatch
            running_loss = 0.0
            
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch

    if(np.mean(ep_lossi)<=0.001):
      break
print('Finished Training')

In [260]:
correct = 0
total = 0
datx = []
labelsx = []
with torch.no_grad():
    for data in train_loader:
        images, labels,_ = data
        
        batch = images.size(0)
        patches = images.size(1)
        
        images = torch.reshape(images,(batch*patches,2))
        labels = labels.repeat_interleave(patches)
        datx.append(images)
        labelsx.append(labels)
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted.shape)
        #print(labels.shape)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the train images: %d %%' % ( 100 * correct / total))

In [261]:
datx = np.concatenate(datx,axis=0)
labelsx = np.concatenate(labelsx,axis=0)
print(datx.shape,labelsx.shape) 

In [262]:

X,Y = torch.meshgrid(torch.linspace(-7, 7,100),torch.linspace(-6, 9, 100))



n=X.shape[0]*X.shape[1]
data = torch.zeros((n,2)).double()
data[:,0]=X.reshape((-1,))
data[:,1]=Y.reshape((-1,))
data = torch.tensor(data).double()
data = data.to("cuda")
  
  
with torch.no_grad():
  Z1 = net(data)
  #print(Z1.shape)
_, Z1 = torch.max(Z1.data, 1)

  #print(Z1.shape)
Z1 = Z1.cpu().numpy()
    
Z1 = Z1.reshape(X.shape)

print(np.unique(Z1))
plt.figure(figsize=(6,8))
cax = plt.contourf(X,Y,Z1)
plt.colorbar(cax)

scatter = plt.scatter(datx[:,0] ,datx[:,1],c=labelsx,cmap=plt.cm.Paired,alpha=0.8)
#plt.yticks(()) #Y value is meaningless in this data. 
plt.legend(handles=scatter.legend_elements()[0], labels=["fg0","fg1","fg2"])
plt.title("classification map "+str(1000))

# Test Data

In [263]:
np.random.seed(2)

y_test = np.concatenate((np.zeros(500),np.ones(500),np.ones(500)*2))
#y = np.random.randint(0,3,6000)
idx_t= []
for i in range(3):
    print(i,sum(y_test==i))
    idx_t.append(y_test==i)
x_test = np.zeros((1500,2))


np.random.seed(13)
x_test[idx_t[0],:] = np.random.multivariate_normal(mean = [5,5],cov=[[0.1,0],[0,0.1]],size=sum(idx_t[0]))
x_test[idx_t[1],:] = np.random.multivariate_normal(mean = [-6,7],cov=[[0.1,0],[0,0.1]],size=sum(idx_t[1]))
x_test[idx_t[2],:] = np.random.multivariate_normal(mean = [-5,-4],cov=[[0.1,0],[0,0.1]],size=sum(idx_t[2]))

In [264]:
x_test= (x_test - mean_x)/std_x

In [265]:
for i in range(3):
    plt.scatter(x_test[idx_t[i],0],x_test[idx_t[i],1],label="class_"+str(i))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [266]:
x_test = torch.tensor(x_test).to("cuda")
outputs = net(x_test)
_, predicted = torch.max(outputs.data, 1)
print("Accuracy on True Data ", np.sum(predicted.cpu().numpy()  == y_test) /15,"%")